In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 28:31].values
classes = np.reshape(classes, (2000, 700, 3))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 3)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM, Bidirectional, Masking

In [6]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
  
    model.add(Dense(3, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [7]:
accu = []
precisions = []
recalls = []
f1 = []

In [8]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 3))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 3))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accu.append(balanced_accuracy_score(y_test, predicted)) 
    precisions.append(precision_score(y_test, predicted, average = 'weighted'))
    recalls.append(recall_score(y_test, predicted, average = 'weighted'))
    f1.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [9]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0819 21:26:19.110342  5592 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0819 21:26:19.115306  5592 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0819 21:26:19.116303  5592 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   1    2    3 ... 1995 1998 1999] TEST: [   0    4   19   21   28   33   39   48   49   54   60   62   63   65
   75   76   84   92   95   99  101  107  108  115  122  130  132  135
  139  146  147  154  161  173  179  181  184  192  195  202  211  216
  221  228  233  241  247  250  257  259  261  266  270  278  281  288
  291  301  302  304  310  319  320  329  336  339  342  350  354  364
  373  380  385  395  399  409  410  417  421  437  440  443  445  446
  447  450  453  460  463  468  469  479  481  487  488  491  494  500
  505  506  509  512  519  532  533  543  544  560  561  565  570  571
  576  581  583  584  590  594  597  600  611  618  622  624  629  632
  637  645  658  663  670  672  687  688  689  695  700  706  707  708
  720  725  733  736  738  740  747  757  758  762  772  775  776  777
  784  794  800  817  831  833  850  856  857  859  869  876  879  880
  883  884  899  900  906  917  918  920  921  928  929  940  942  948
  951  956  958  961  963  9

W0819 21:26:20.116643  5592 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 10s 6ms/sample - loss: 0.3566 - acc: 0.7689
Epoch 2/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3190 - acc: 0.8534
Epoch 3/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3049 - acc: 0.8622
Epoch 4/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3003 - acc: 0.8653
Epoch 5/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2956 - acc: 0.8680
Epoch 6/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2936 - acc: 0.8690
Epoch 7/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2921 - acc: 0.8697
Epoch 8/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2874 - acc: 0.8718
Epoch 9/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2867 - acc: 0.8721
Epoch 10/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2860 - acc: 0.872

1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1310 - acc: 0.9449
Epoch 83/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1291 - acc: 0.9455
Epoch 84/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1273 - acc: 0.9465
Epoch 85/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1263 - acc: 0.9469
Epoch 86/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1270 - acc: 0.9465
Epoch 87/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1273 - acc: 0.9465
Epoch 88/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1245 - acc: 0.9478
Epoch 89/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1258 - acc: 0.9469
Epoch 90/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1260 - acc: 0.9466
Epoch 91/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1221 - acc: 0.9488
Epo

...

Epoch 1/100
1600/1600 [==============================] - 9s 6ms/sample - loss: 0.3601 - acc: 0.7831
Epoch 2/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3212 - acc: 0.8526
Epoch 3/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3075 - acc: 0.8609
Epoch 4/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3018 - acc: 0.8576
Epoch 5/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2992 - acc: 0.8660
Epoch 6/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2954 - acc: 0.8683
Epoch 7/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2906 - acc: 0.8699
Epoch 8/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2896 - acc: 0.8702
Epoch 9/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2891 - acc: 0.8697
Epoch 10/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2855 - acc: 0.8729

1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1270 - acc: 0.9443
Epoch 83/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1256 - acc: 0.9452
Epoch 84/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1258 - acc: 0.9450
Epoch 85/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1238 - acc: 0.9458
Epoch 86/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1231 - acc: 0.9460
Epoch 87/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1223 - acc: 0.9468
Epoch 88/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1214 - acc: 0.9458
Epoch 89/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1210 - acc: 0.9469
Epoch 90/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1210 - acc: 0.9475
Epoch 91/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1205 - acc: 0.9471
Epo

1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2153 - acc: 0.9040
Epoch 38/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2135 - acc: 0.9049
Epoch 39/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2113 - acc: 0.9054
Epoch 40/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2072 - acc: 0.9067
Epoch 41/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2046 - acc: 0.9077
Epoch 42/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2015 - acc: 0.9083
Epoch 43/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1984 - acc: 0.9090
Epoch 44/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1957 - acc: 0.9094
Epoch 45/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1959 - acc: 0.9103
Epoch 46/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1938 - acc: 0.9101
Epo

Epoch 1/100
1600/1600 [==============================] - 9s 6ms/sample - loss: 0.3610 - acc: 0.8140
Epoch 2/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3222 - acc: 0.8523
Epoch 3/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3090 - acc: 0.8614
Epoch 4/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.3048 - acc: 0.8638
Epoch 5/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2981 - acc: 0.8673
Epoch 6/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2967 - acc: 0.8680
Epoch 7/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2918 - acc: 0.8705
Epoch 8/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2903 - acc: 0.8710
Epoch 9/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2897 - acc: 0.8710
Epoch 10/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.2862 - acc: 0.8724

1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1492 - acc: 0.9372
Epoch 83/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1475 - acc: 0.9377
Epoch 84/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1473 - acc: 0.9378
Epoch 85/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1461 - acc: 0.9382
Epoch 86/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1441 - acc: 0.9385
Epoch 87/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1443 - acc: 0.9389
Epoch 88/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1429 - acc: 0.9392
Epoch 89/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1409 - acc: 0.9406
Epoch 90/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1416 - acc: 0.9398
Epoch 91/100
1600/1600 [==============================] - 8s 5ms/sample - loss: 0.1413 - acc: 0.9403
Epo

In [10]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_12 (Bidirectio (None, 700, 200)          97600     
_________________________________________________________________
dropout_12 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_13 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_13 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_14 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
dense_4 (Dense)              (None, 700, 3)           

In [12]:
print('Acurácias total')
print(accu)
accu = np.array(accu)
print(accu.mean())
print('Precision total')
print(precisions)
precisions = np.array(precisions)
print(precisions.mean())
print('Recalls total')
print(recalls)
recalls = np.array(recalls)
print(recalls.mean())
print('F1 total')
print(f1)
f1 = np.array(f1)
print(f1.mean())

Acurácias total
[0.67849544 0.66855661 0.66863772 0.66271654 0.66818567]
0.6693183973771628
Precision total
[0.69461045 0.68405037 0.68584642 0.67568604 0.68881973]
0.6858026022983281
Recalls total
[0.69503014 0.68406249 0.68697795 0.67609511 0.68907613]
0.6862483639663141
F1 total
[0.69353331 0.68298742 0.68583829 0.67489247 0.68720505]
0.6848913068901454
